# Lê Quang Phú
# 2174802010514

### Câu 1 (2 điểm): Cho một ảnh bất kỳ (tên ảnh do sinh viên tự đặt, ví dụ: `my_image.jpg`) và thực hiện các yêu cầu sau:

* Viết chương trình sử dụng Bilateral filter để làm mịn ảnh. (0.5 điểm)  
* Viết chương trình sử dụng Canny Edge Detection để xác định biên của hình ảnh. (0.5 điểm)  
* Đổi màu ảnh bằng cách hoán đổi kênh màu theo thứ tự (ví dụ: BGR → BRG) và lưu thành tên dạng `[ten_anh]_swapped.jpg`. (0.5 điểm)  
* Chuyển ảnh sang không gian màu YCrCb và tách riêng 3 kênh Y, Cr, Cb, lưu thành ảnh grayscale tương ứng (`[ten_anh]_Y.jpg`, `[ten_anh]_Cr.jpg`, `[ten_anh]_Cb.jpg`). (0.5 điểm)


In [ ]:
import cv2
import os

image_path = 'my_image.jpg'  
image_name = os.path.splitext(os.path.basename(image_path))[0]

img = cv2.imread(image_path)

bilateral = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
cv2.imwrite(f"{image_name}_bilateral.jpg", bilateral)
edges = cv2.Canny(img, threshold1=100, threshold2=200)
cv2.imwrite(f"{image_name}_canny.jpg", edges)
swapped = img.copy()
swapped[:, :, [1, 2]] = swapped[:, :, [2, 1]]  
cv2.imwrite(f"{image_name}_swapped.jpg", swapped)
ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
Y, Cr, Cb = cv2.split(ycrcb)
cv2.imwrite(f"{image_name}_Y.jpg", Y)
cv2.imwrite(f"{image_name}_Cr.jpg", Cr)
cv2.imwrite(f"{image_name}_Cb.jpg", Cb)

print("Hoàn thành xử lý ảnh. Các ảnh đã được lưu.")


Hoàn thành xử lý ảnh. Các ảnh đã được lưu.


### Câu 2 (4 điểm) Viết một chương trình Python sử dụng OpenCV để tạo menu tương tác cho phép người dùng chọn các kỹ thuật biến đổi hình học và xử lý ảnh nâng cao từ một danh sách, áp dụng đồng thời cho nhiều ảnh.

### Yêu cầu:

1. Menu gồm:  
* Phóng to ảnh (Zoom bằng resize) (0.5 điểm)  
* Xoay ảnh (góc ngẫu nhiên từ 0–360 độ) (0.5 điểm)  
* Lật ảnh ngang (0.5 điểm)  
* Lật ảnh dọc (0.5 điểm)  
* Cắt ảnh (crop ngẫu nhiên vùng giữa ảnh) (0.5 điểm)  
* Thêm viền (padding màu ngẫu nhiên) (0.5 điểm)

2. Chương trình xử lý đồng thời 3 ảnh bất kỳ do sinh viên tự chọn (có thể chọn bằng đường dẫn file hoặc nhập tên ảnh tùy ý). (0.5 điểm)

3. Phím tương ứng để kích hoạt các phương pháp xử lý:  
* Z: Zoom  
* T: Rotate  
* H: Horizontal Flip  
* V: Vertical Flip  
* C: Crop  
* P: Padding (0.5 điểm)

4. Lưu file kết quả với định dạng: `result_[phương pháp]_[tên ảnh gốc].jpg`  
   Ví dụ: `result_crop_cat.jpg`, `result_rotate_image1.jpg` (0.5 điểm)


In [1]:
import cv2
import os
import random
import numpy as np
from pathlib import Path

def zoom(img, scale=1.5):
    return cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
def rotate(img):
    angle = random.randint(0, 360)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    return cv2.warpAffine(img, M, (w, h))
def horizontal_flip(img):
    return cv2.flip(img, 1)
def vertical_flip(img):
    return cv2.flip(img, 0)
def crop(img):
    h, w = img.shape[:2]
    top = random.randint(0, h // 4)
    left = random.randint(0, w // 4)
    bottom = random.randint(3 * h // 4, h)
    right = random.randint(3 * w // 4, w)
    return img[top:bottom, left:right]
def padding(img):
    top, bottom = random.randint(10, 50), random.randint(10, 50)
    left, right = random.randint(10, 50), random.randint(10, 50)
    color = [random.randint(0, 255) for _ in range(3)]
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

operations = {
    'Z': ("zoom", zoom),
    'T': ("rotate", rotate),
    'H': ("horizontal", horizontal_flip),
    'V': ("vertical", vertical_flip),
    'C': ("crop", crop),
    'P': ("padding", padding)
}
image_paths = input("Nhập đường dẫn các ảnh, cách nhau bằng dấu phẩy: ").split(",")
image_paths = [img.strip() for img in image_paths]

print("\nMenu thao tác:")
print("Z: Zoom")
print("T: Rotate")
print("H: Horizontal Flip")
print("V: Vertical Flip")
print("C: Crop")
print("P: Padding")

keys = input("Nhập các ký tự tương ứng với các thao tác muốn áp dụng (VD: ZTP): ").upper()

for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is None:
        print(f"Lỗi đọc ảnh: {img_path}")
        continue

    name = Path(img_path).stem
    for key in keys:
        if key in operations:
            method_name, func = operations[key]
            out_img = func(img)
            out_path = f"result_{method_name}_{name}.jpg"
            cv2.imwrite(out_path, out_img)
            print(f"✔ Đã lưu: {out_path}")
        else:
            print(f"❌ Ký tự không hợp lệ: {key}")



Menu thao tác:
Z: Zoom
T: Rotate
H: Horizontal Flip
V: Vertical Flip
C: Crop
P: Padding
✔ Đã lưu: result_zoom_bird.jpg
✔ Đã lưu: result_rotate_bird.jpg
✔ Đã lưu: result_horizontal_bird.jpg
✔ Đã lưu: result_zoom_fruit.jpg
✔ Đã lưu: result_rotate_fruit.jpg
✔ Đã lưu: result_horizontal_fruit.jpg
✔ Đã lưu: result_zoom_my_image.jpg
✔ Đã lưu: result_rotate_my_image.jpg
✔ Đã lưu: result_horizontal_my_image.jpg


### Câu 3 (4 điểm) Viết một chương trình Python để xử lý 3 ảnh bất kỳ do sinh viên tự chọn.

* Thêm viền đen 20 pixel cho ảnh đầu tiên. (0.5 điểm)  
* Xoay ảnh thứ hai 45 độ và phóng to 1.5 lần. (0.5 điểm)  
* Tăng kích thước ảnh thứ ba lên 4 lần, sau đó áp dụng Bilateral Filter với tham số tùy chọn. (1.5 điểm)  
* Thay đổi độ sáng và độ tương phản ảnh thứ ba theo công thức:

$$
I_{out}(x, y) = \alpha \cdot I_{in}(x, y) + \beta
$$

Trong đó:  

$$
\alpha \in [0.6, 2.0], \quad \beta \in [-60, 60]
$$

Giá trị đầu ra cần được giới hạn trong khoảng [0, 255] bằng công thức:

$$
I_{out}(x, y) = \text{clip}(I_{out}(x, y), 0, 255)
$$


# Chúc các bạn thi may mắn và đạt điểm 10.

In [2]:
import cv2
import numpy as np
import random
import os

def add_padding(img, pad=20):
    return cv2.copyMakeBorder(img, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=(0, 0, 0))
def rotate_and_scale(img, angle=45, scale=1.5):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    return cv2.warpAffine(img, M, (int(w * scale), int(h * scale)))
def resize_and_bilateral(img, scale=4):
    resized = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    return cv2.bilateralFilter(resized, d=9, sigmaColor=75, sigmaSpace=75)
def adjust_brightness_contrast(img, alpha=None, beta=None):
    if alpha is None:
        alpha = round(random.uniform(0.6, 2.0), 2)
    if beta is None:
        beta = random.randint(-60, 60)
    print(f"→ Điều chỉnh độ sáng/tương phản với alpha={alpha}, beta={beta}")
    return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
def process_images(image_paths):
    for idx, path in enumerate(image_paths):
        img = cv2.imread(path)
        if img is None:
            print(f"❌ Không đọc được ảnh: {path}")
            continue
        name = os.path.splitext(os.path.basename(path))[0]
        if idx == 0:
            out = add_padding(img)
            cv2.imwrite(f"output_{name}_padding.jpg", out)
            print(f"✔ Đã lưu: output_{name}_padding.jpg")
        elif idx == 1:
            out = rotate_and_scale(img)
            cv2.imwrite(f"output_{name}_rotate_scale.jpg", out)
            print(f"✔ Đã lưu: output_{name}_rotate_scale.jpg")
        elif idx == 2:
            out = resize_and_bilateral(img)
            cv2.imwrite(f"output_{name}_resize_bilateral.jpg", out)
            print(f"✔ Đã lưu: output_{name}_resize_bilateral.jpg")
        bright_contrast = adjust_brightness_contrast(img)
        cv2.imwrite(f"output_{name}_brightness_contrast.jpg", bright_contrast)
        print(f"✔ Đã lưu: output_{name}_brightness_contrast.jpg")
if __name__ == "__main__":
    paths = input("Nhập đường dẫn 3 ảnh, cách nhau bởi dấu phẩy: ").split(",")
    paths = [p.strip() for p in paths][:3]
    process_images(paths)


✔ Đã lưu: output_bird_padding.jpg
→ Điều chỉnh độ sáng/tương phản với alpha=1.57, beta=-7
✔ Đã lưu: output_bird_brightness_contrast.jpg
✔ Đã lưu: output_fruit_rotate_scale.jpg
→ Điều chỉnh độ sáng/tương phản với alpha=1.82, beta=14
✔ Đã lưu: output_fruit_brightness_contrast.jpg
✔ Đã lưu: output_my_image_resize_bilateral.jpg
→ Điều chỉnh độ sáng/tương phản với alpha=0.6, beta=-51
✔ Đã lưu: output_my_image_brightness_contrast.jpg
